In [1]:
%run basics
%matplotlib
import xlrd
from matplotlib.patches import Polygon

Using matplotlib backend: Qt4Agg


In [2]:
def get_data_xlsheet(sheet,frow,label):
    header = sheet.row_values(frow-1)
    col = header.index(label)
    lrow = int(sheet.nrows)
    a = numpy.array(active_sheet.col_values(col)[frow:lrow])
    idx = numpy.where(~numpy.isnan(a)&(a!=-9999))[0]
    return a[idx]

In [3]:
def colour_boxes(ax,bp,data,nsites,nalts):
    boxColors = {1:['darkkhaki'],2:['darkkhaki','royalblue'],
                 3:['red','darkkhaki','royalblue']}
    numBoxes = nsites*nalts
    medians = list(range(numBoxes))
    for i in range(numBoxes):
        box = bp['boxes'][i]
        boxX = []
        boxY = []
        for j in range(5):
            boxX.append(box.get_xdata()[j])
            boxY.append(box.get_ydata()[j])
        boxCoords = list(zip(boxX, boxY))
        # Alternate between Dark Khaki and Royal Blue
        k = i % nalts
        boxPolygon = Polygon(boxCoords, facecolor=boxColors[nalts][k])
        ax.add_patch(boxPolygon)
        # Now draw the median lines back over what we just filled in
        med = bp['medians'][i]
        medianX = []
        medianY = []
        for j in range(2):
            medianX.append(med.get_xdata()[j])
            medianY.append(med.get_ydata()[j])
            ax.plot(medianX, medianY, 'k')
            medians[i] = medianY[0]

In [4]:
def create_site_dict(FluxNet_ID):
    d = {"FluxNet_ID":FluxNet_ID,
        "Fsd":{"access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
               "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Fld":{"access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
               "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Fn":{"access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Fg":{"access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Sws":{"access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
               "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Ts":{"access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Ta":{"aws":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Ah":{"aws":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}},
        "Ws":{"aws":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "access":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]},
              "erai":{"RMSE":[],"r":[],"Var ratio":[],"Bias":[]}}}
    return d

In [5]:
# make a dictionary of sites ...
sites = {"Calperum":{"FluxNet_ID":"AU-Cpr"},
         "CumberlandPlains":{"FluxNet_ID":"AU-Cum"},
         "DalyUncleared":{"FluxNet_ID":"AU-DaS"},
         "Gingin":{"FluxNet_ID":"AU-Gin"},
         "HowardSprings":{"FluxNet_ID":"AU-How"},
         "SturtPlains":{"FluxNet_ID":"AU-Stp"},
         "Tumbarumba":{"FluxNet_ID":"AU-Tum"},
         "Whroo":{"FluxNet_ID":"AU-Whr"},
         "WombatStateForest":{"FluxNet_ID":"AU-Wom"}}
# ... and a dictionary for the data
data = {}
for site in sites.keys():
    data[site] = create_site_dict(sites[site]["FluxNet_ID"])

In [6]:
site_name_list = data.keys()
site_name_list.sort()
nsites = len(site_name_list)

In [8]:
# read the data from the L4 summary spreadsheet
frow = 10
for site in site_name_list:
    xl_path = "../Docs/SpecialIssue_2014/OzFluxDataPath/Data/L4_statistics/"
    xl_name = xl_path+site+"_L4_AlternateStats.xls"
    xl_book = xlrd.open_workbook(xl_name)
    # get data from access and erai
    for var in ["Fsd","Fld","Fn","Fg","Sws","Ts"]:
        for alt in ["access","erai"]:
            sheet_name = var+"_"+alt
            active_sheet = xl_book.sheet_by_name(sheet_name)
            for item in ["RMSE","r","Var ratio","Bias"]:
                data[site][var][alt][item] = get_data_xlsheet(active_sheet,frow,item)
    # get data from aws, access and erai
    for var in ["Ta","Ah","Ws"]:
        for alt in ["aws","access","erai"]:
            sheet_name = var+"_"+alt
            active_sheet = xl_book.sheet_by_name(sheet_name)
            for item in ["RMSE","r","Var ratio","Bias"]:
                data[site][var][alt][item] = get_data_xlsheet(active_sheet,frow,item)

In [9]:
# Fsd plot
# shuffle the data
variables = {"Fsd":{"title":"Incoming shortwave radiation",
                    "units":"W/m2",
                    "alt_list":["access","erai"]},
             "Fld":{"title":"Incoming longwave radiation",
                    "units":"W/m2",
                    "alt_list":["access","erai"]},
             "Fn":{"title":"Net radiation",
                   "units":"W/m2",
                   "alt_list":["access","erai"]},
             "Fg":{"title":"Ground heat flux",
                   "units":"W/m2",
                   "alt_list":["access","erai"]},
             "Sws":{"title":"Soil water content",
                    "units":"frac",
                    "alt_list":["access","erai"]},
             "Ts":{"title":"Soil temperature",
                   "units":"C",
                   "alt_list":["access","erai"]},
             "Ta":{"title":"Air Temperature",
                    "units":"C",
                    "alt_list":["aws","access","erai"]},
             "Ah":{"title":"Absolute humidity",
                    "units":"g/m3",
                    "alt_list":["aws","access","erai"]},
             "Ws":{"title":"Wind speed",
                    "units":"m/s",
                    "alt_list":["aws","access","erai"]}}
boxprops = dict(linestyle='-',color='black')
for var in variables.keys():
    rmse = []; r =[]; var_ratio = []; bias = []
    xlabels_site = []
    nalts = len(variables[var]["alt_list"])
    minor_ticks = [x+0.5 for x in range(nalts,nalts*nsites,nalts)]
    for site in site_name_list:
        for alt in variables[var]["alt_list"]:
            xlabels_site.append(data[site]["FluxNet_ID"])
            rmse.append(data[site][var][alt]["RMSE"])
            r.append(data[site][var][alt]["r"])
            var_ratio.append(data[site][var][alt]["Var ratio"])
            bias.append(data[site][var][alt]["Bias"])
    # do the plot
    xlabels_site = ["" if i%nalts!=0 else x for i,x in enumerate(xlabels_site)]
    fig,axs = plt.subplots(4,figsize=(8,6))
    fig.canvas.set_window_title(var)
    plt.subplots_adjust(hspace=0.15)
    axs[0].set_title(variables[var]["title"])
    rmse_bp = axs[0].boxplot(rmse,boxprops=boxprops,whis=0,sym="",showcaps=False)
    ylabel = "RMSE ("+variables[var]["units"]+")"
    axs[0].set_ylabel(ylabel)
    colour_boxes(axs[0],rmse_bp,rmse,nsites,nalts)
    axs[0].set_xticks(minor_ticks, minor=True)
    axs[0].xaxis.grid(which='minor')

    r_bp = axs[1].boxplot(r,boxprops=boxprops,whis=0,sym="",showcaps=False)
    axs[1].set_ylabel("R")
    colour_boxes(axs[1],r_bp,r,nsites,nalts)
    axs[1].set_xticks(minor_ticks, minor=True)
    axs[1].xaxis.grid(which='minor')

    vr_bp = axs[2].boxplot(var_ratio,boxprops=boxprops,whis=0,sym="",showcaps=False)
    axs[2].set_ylabel("Var ratio")
    colour_boxes(axs[2],vr_bp,var_ratio,nsites,nalts)
    axs[2].set_xticks(minor_ticks, minor=True)
    axs[2].xaxis.grid(which='minor')
    axs[2].axhline(y=1,linewidth=2,linestyle=':',color='red',alpha=0.5)

    bias_bp = axs[3].boxplot(bias,boxprops=boxprops,whis=0,sym="",showcaps=False)
    ylabel = "Bias ("+variables[var]["units"]+")"
    axs[3].set_ylabel(ylabel)
    colour_boxes(axs[3],bias_bp,bias,nsites,nalts)
    axs[3].set_xticks(minor_ticks, minor=True)
    axs[3].xaxis.grid(which='minor')
    axs[3].axhline(y=0,linewidth=2,linestyle=':',color='red',alpha=0.5)

    xticklabels = axs[0].get_xticklabels()+axs[1].get_xticklabels()+axs[2].get_xticklabels()
    plt.setp(xticklabels, visible=False)
    xtickNames = plt.setp(axs[3], xticklabels=xlabels_site)
    plt.setp(xtickNames, rotation=45, fontsize=10)
    plt.show()

In [11]:
rmse

[array([ 4.56573115,  2.955982  ,  1.69196058,  2.00811912,  2.72531949,
         2.33793482,  1.84238241,  3.36248049,  6.82591128,  5.84956248,
         1.44739853,  1.71598495,  5.6107423 ,  4.93115781,  3.10375672,
         1.83282935,  7.92637749,  3.05362295,  3.19088005,  4.04133971,
         7.00816829]),
 array([ 3.2279335 ,  4.51556435,  3.50490518,  2.62316635,  3.25468923,
         2.84317354,  2.29560316,  2.49728351,  3.76014471,  5.81774942,
         5.20596119,  1.65726209,  2.04815844,  5.64949341,  4.92863075,
         3.76867121,  2.49400885,  7.68234629,  3.05850409,  3.7635479 ,
         4.6730614 ]),
 array([ 1.10553616,  1.31095346,  0.97209675,  2.1087386 ,  2.493739  ,
         2.35038586,  2.65666286,  2.26820647,  1.11105083,  3.22709669,
         1.16225716,  2.40468912,  2.46611745]),
 array([ 1.19795651,  3.78734085,  1.39573449,  3.43671077,  4.13517956,
         3.78940156,  3.73955115,  3.80735903,  1.18955018,  3.7851036 ,
         1.32184413,  3.23978